## Create a calibration using pyFAI

In [ ]:
import matplotlib.pyplot as plt
import pyFAI.gui.jupyter
import pyFAI
import fabio
from pyFAI.test.utilstest import UtilsTest
from pyFAI.calibrant import CALIBRANT_FACTORY
from pyFAI.goniometer import SingleGeometry
import pathlib
from tqdm import tqdm
import warnings
figure_size = 10

Load and show the calibration image

In [ ]:
filename = "example-calibration/DLS_CeO2_1200mm.tif"
frame = fabio.open(filename).data
figure = plt.figure(figsize=(figure_size, figure_size))
ax = plt.gca()
pyFAI.gui.jupyter.display(frame, ax=ax)
print("Note, this image appears mirrored from top to bottom, as the origin is at the bottom left.")

Define geometry components such as beam center and detector distance.

In [ ]:
x = 712.136 # x-coordinate of the beam-center in pixels
y = 727.864 # y-coordinate of the beam-center in pixels
d = 1194.046 # This is the distance in mm (unit used by Fit2d)
wl = 1.393000e-11 # Beam wavelength

Definition of the detector and of the calibrant

In [ ]:
detector = pyFAI.detectors.Detector(pixel1=0.000296, pixel2=0.000296)
print(detector)
calibrant = CALIBRANT_FACTORY("CeO2")
calibrant.wavelength = wl
calibrant

Set the geometry of the detector to be the guessed geometry

In [ ]:
initial_geometry = pyFAI.geometry.Geometry(detector=detector, wavelength=wl)
initial_geometry.setFit2D(d, x, y)
initial_geometry

We now use the SingleGeometry object to perform automatic ring extraction and calibration

In [ ]:
sg = SingleGeometry("demo", frame, calibrant=calibrant, detector=detector, geometry=initial_geometry)
sg.extract_cp(max_rings=16)

This figure shows that the fitted control points and rings overlap pretty well. Refinement probably won't change much.

In [ ]:
figure = plt.figure(figsize=(figure_size, figure_size))
ax = plt.gca()
pyFAI.gui.jupyter.display(sg=sg, ax=ax)

We refine the geometry, we leave the wavelength fixed as we know this, but allow the other parameters to change. This accounts for any slight inaccuracy in the detector set up.

In [ ]:
sg.geometry_refinement.refine2(fix=["wavelength"])

This figure shows the result of the refinement.

In [ ]:
figure = plt.figure(figsize=(figure_size, figure_size))
ax = plt.gca()
pyFAI.gui.jupyter.display(sg=sg, ax=ax)

We can now save the refinement to a file to be used later

In [ ]:
# Delete the calibration if it already exists
calibration_path = "example-calibration/DLS_CeO2_1200mm_pyFAI.poni"
# pathlib.Path(calibration_path).unlink(missing_ok=True) # only works for 3.8+
try:
    pathlib.Path(calibration_path).unlink()
except FileNotFoundError:
    pass

#Save the geometry obtained
sg.geometry_refinement.save(calibration_path)
with open(calibration_path) as f:
    print(f.read())